In [27]:
import numpy as np
import math
from sklearn.model_selection import train_test_split
from data import *
from evaluation import *
%matplotlib inline

### 加载数据集

In [16]:
# data_dir = 'datasets/ml-1m/ratings.dat' 
data_dir='datasets/ml-100k/u.data'
N, M, data_list, _ = load_data(file_dir=data_dir)
print(' data length: %d \n user number: %d \n item number: %d' %(len(data_list),N,M))

### 分割数据集

In [17]:
train_list, test_list = train_test_split(data_list,test_size=0.2)
print ('train length: %d \n test length: %d' %(len(train_list),len(test_list)))

### 将 list 转换成 矩阵

In [18]:
train_mat = sequence2mat(sequence = train_list, N = N, M = M)
test_mat = sequence2mat(sequence = test_list, N = N, M = M)

### 参数设置

In [19]:
K = 20
# lambda
lamda_regularizer = 0.1
# 学习率
learning_rate = 0.005
max_iteration = 100

### 概率矩阵分解 （PMF）

In [20]:
from pmf import *
import time
model = pmf(train_list=train_list, 
            test_list=test_list, 
            N=N, 
            M=M,
            K=K,
            P=None,
            Q=None,
            learning_rate=learning_rate, 
            lamda_regularizer=lamda_regularizer, 
            max_iteration=max_iteration)
P, Q, records_array ,during= model.train()
print('MAE:%.4f;RMSE:%.4f;Recall:%.4f;Precision:%.4f;F-value:%.4f'
      %(records_array[:,1][-1],records_array[:,2][-1],records_array[:,3][-1],records_array[:,4][-1],((2*records_array[:,3][-1]*records_array[:,4][-1])/(records_array[:,3][-1]+records_array[:,4][-1]))))
print(during)



In [21]:
import numpy as np
from evaluation import *
k5_records=np.array(records_array)
np.savetxt('lambda02.txt',k5_records)

# print(records_array)
import matplotlib as plt
plt.rcParams['font.sans-serif'] = ['Songti SC']
plt.rcParams['font.size'] = 10.5

# records_array=np.loadtxt('lambda01.txt')

figure(values_list=records_array[:,0],name='loss',color='r',)
figure(values_list=records_array[:,1],name='MAE',color='r')
figure(values_list=records_array[:,2],name="rmse",color='r')

In [22]:
import matplotlib.pyplot as plt

# 设置宋体字体
plt.rcParams['font.sans-serif'] = ['Songti SC']
plt.rcParams['font.size'] = 10.5

# 数据
x = [1, 2, 3, 4, 5]
y = [2, 3, 5, 7, 6]

# 绘图
plt.plot(x, y)
plt.xlabel('Iterations', fontdict={'family': 'sans-serif'})
plt.ylabel('Iterations', fontdict={'family': 'sans-serif'})
plt.title('loss curve', fontdict={'family': 'sans-serif'})
plt.show()


In [23]:
import pickle

# 保存模型
with open('pmf_model-100k.pkl', 'wb') as f:
    pickle.dump(model, f)

In [24]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

# 创建图形对象和3D子图
fig = plt.figure(figsize=(10, 10),dpi=200)
iterations = np.arange(1, 101)
iterations001 = np.arange(1, 25)
iterations005 = np.arange(1, 55)

ax = fig.add_subplot(111, projection='3d')

# 读取数据
rmse_k10 = np.loadtxt('k10_records.txt')
rmse_k15 = np.loadtxt('k15_records.txt')
rmse_k20 = np.loadtxt('k20.txt')
rmse_k25 = np.loadtxt('k25.txt')

#绘制曲线并手动添加图例
ax.plot(iterations, rmse_k10[:,2], zs=0.001, zdir='y', color='blue', linestyle='-', marker='o')
# ax.plot(iterations005, rmse_k15[:,0], zs=15, zdir='y', color='green', linestyle='-', marker='s')
ax.plot(iterations, rmse_k20[:,2], zs=0.005, zdir='y', color='red', linestyle='-', marker='^')
ax.plot(iterations, rmse_k25[:,2], zs=0.01, zdir='y', color='cyan', linestyle='-', marker='o')

# #绘制曲线并手动添加图例
# ax.plot(iterations, rmse_k10[:,0], zs=10, zdir='y', color='blue', linestyle='-', marker='o')
# ax.plot(iterations, rmse_k15[:,0], zs=15, zdir='y', color='green', linestyle='-', marker='s')
# ax.plot(iterations, rmse_k20[:,0], zs=20, zdir='y', color='red', linestyle='-', marker='^')
# ax.plot(iterations, rmse_k25[:,0], zs=25, zdir='y', color='cyan', linestyle='-', marker='o')


# 手动创建图例
handles = [plt.Line2D([0], [0], color='blue', linestyle='-', marker='o', label='学习率=0.001'),
        #    plt.Line2D([0], [0], color='green', linestyle='-', marker='s', label='学习率=0.05'),
           plt.Line2D([0], [0], color='red', linestyle='-', marker='^', label='学习率=0.005'),
           plt.Line2D([0], [0], color='cyan', linestyle='-', marker='o', label='学习率=0.01')]

# # 手动创建图例
# handles = [plt.Line2D([0], [0], color='blue', linestyle='-', marker='o', label='K=10'),
#            plt.Line2D([0], [0], color='green', linestyle='-', marker='s', label='K=15'),
#            plt.Line2D([0], [0], color='red', linestyle='-', marker='^', label='K=20'),
#            plt.Line2D([0], [0], color='cyan', linestyle='-', marker='o', label='K=25')]

# 设置宋体字体
plt.rcParams['font.sans-serif'] = ['Songti SC']
plt.rcParams['font.size'] = 15

# 设置轴标签
ax.set_xlabel('Iterations', fontdict={'family': 'sans-serif'},labelpad=10,size=15)
ax.set_ylabel(' 学习率取值', fontdict={'family': 'sans-serif'},labelpad=10,size=15)
ax.set_zlabel('RMSE', fontdict={'family': 'sans-serif'},labelpad=10,size=15)

# 添加手动创建的图例
ax.legend(handles=handles)

# 调整视角
ax.view_init(elev=30, azim=68)

# 显示图表
plt.show()


In [25]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

# 创建图形对象和 3D 子图
fig = plt.figure(figsize=(20, 10))
ax = fig.add_subplot(111, projection='3d')

# 读取数据
iterations = np.arange(1, 101)
rmse_k10 = np.loadtxt('k10_records.txt')
rmse_k15 = np.loadtxt('k15_records.txt')
rmse_k20 = np.loadtxt('k20.txt')
rmse_k25 = np.loadtxt('k25.txt')

# 创建匹配的坐标网格
X, Y = np.meshgrid(iterations, [10, 15, 20, 25])

# 绘制曲线的散点图并手动添加图例
ax.scatter(X, Y, rmse_k10, color='blue', label='K=10', marker='o')
ax.scatter(X, Y, rmse_k15, color='green', label='K=15', marker='s')
ax.scatter(X, Y, rmse_k20, color='red', label='K=20', marker='^')
ax.scatter(X, Y, rmse_k25, color='cyan', label='K=25', marker='o')

# 设置轴标签
ax.set_xlabel('Iterations')
ax.set_ylabel('K value')
ax.set_zlabel('RMSE')

# 设置图例
ax.legend()

# 调整视角
ax.view_init(elev=30, azim=50)

# 显示图表
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# 创建数据
x = [1, 2, 3, 4, 5]
y = [2, 4, 6, 8, 10]
z = [1, 3, 5, 7, 9]

# 创建图形对象和坐标轴
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# 绘制曲线并设置图例标签
ax.plot(x, y, z, label='曲线1')
ax.plot(y, x, z, label='曲线2')

# 添加图例
ax.legend()

# 显示图形
plt.show()

In [26]:
import pickle
# 加载模型
with open('pmf_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)
# pred_mat = loaded_model.prediction(loaded_model.P, loaded_model.Q)
# recommendations = loaded_model.recommendation(pred_mat, num_recommendations=5)
# print(recommendations)

print(loaded_model.P)
r_pred = loaded_model.prediction(loaded_model.P, loaded_model.Q)  # 调用 prediction 方法获取预测评分矩阵
recommendations = loaded_model.recommendation(r_pred, num_recommendations=10)
print(recommendations)
